In [22]:
using RigidBodyDynamics
using LinearAlgebra
using StaticArrays
using MeshCat
using MeshCatMechanisms
using MechanismGeometries

println("Libraries imported.")

Libraries imported.


In [23]:
# ------------------------------------------------------------------------
#                            MODEL DEFINITION
# ------------------------------------------------------------------------
src_dir = dirname(pathof(RigidBodyDynamics))
urdf_file = joinpath(src_dir, "..", "..", "..", "..", "..", "onr-dynamics-julia", "alphaArmFixedJaws.urdf")
mechanism = parse_urdf(urdf_file)

visuals = URDFVisuals(urdf_file)

println("URDF parsed. \n")

URDF parsed. 



In [24]:
mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf_file))

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://localhost:8703
└ @ MeshCat /home/hkolano/.julia/packages/MeshCat/nXUuG/src/visualizer.jl:69


MechanismVisualizer{MechanismState{Float64,Float64,Float64,TypeSortedCollections.TypeSortedCollection{Tuple{Array{Joint{Float64,Revolute{Float64}},1}},1}},Visualizer}(MechanismState{Float64, Float64, Float64, …}(…), MeshCat Visualizer with path /meshcat at http://localhost:8703, 11)

In [27]:
# ------------------------------------------------------------------------
#                              CONTROLLER
# ------------------------------------------------------------------------
state = MechanismState(mechanism)
zero_velocity!(state)
set_configuration!(state, base_joint, 0.0)
set_configuration!(state, shoulder_joint, 0.0)
set_configuration!(state, elbow_joint, 0.0)
set_configuration!(state, wrist_joint, 0.0)

tau = inverse_dynamics(state, [0., 0., 0., 0.])
print(tau)

base_joint, shoulder_joint, elbow_joint, wrist_joint = joints(mechanism)

function simple_control!(torques::AbstractVector, t, state::MechanismState)
    torques[velocity_range(state, base_joint)] .= 0.1*sin(0.5*t)
    torques[velocity_range(state, shoulder_joint)] .= 0.
    torques[velocity_range(state, elbow_joint)] .= 0.
    torques[velocity_range(state, elbow_joint)] .= 0.
end;

MethodError: [91mMethodError: no method matching inverse_dynamics(::MechanismState{Float64,Float64,Float64,TypeSortedCollections.TypeSortedCollection{Tuple{Array{Joint{Float64,Revolute{Float64}},1}},1}}, ::Array{Float64,1})[39m
[91m[0mClosest candidates are:[39m
[91m[0m  inverse_dynamics(::MechanismState{X,M,C,JointCollection} where JointCollection where C, [91m::SegmentedVector{JointID,V,KeyRange,P} where P<:AbstractArray{V,1} where KeyRange<:AbstractRange{JointID}[39m) where {X, M, V} at /home/hkolano/.julia/packages/RigidBodyDynamics/2qxSc/src/mechanism_algorithms.jl:564[39m
[91m[0m  inverse_dynamics(::MechanismState{X,M,C,JointCollection} where JointCollection where C, [91m::SegmentedVector{JointID,V,KeyRange,P} where P<:AbstractArray{V,1} where KeyRange<:AbstractRange{JointID}[39m, [91m::AbstractDict{BodyID,Wrench{W}}[39m) where {X, M, V, W} at /home/hkolano/.julia/packages/RigidBodyDynamics/2qxSc/src/mechanism_algorithms.jl:564[39m

In [20]:
# ------------------------------------------------------------------------
#                              SIMULATION
# ------------------------------------------------------------------------
final_time = 5.
ts, qs, vs = simulate(state, final_time, simple_control!; Δt = 1e-3)

println("Simulation finished.")

Simulation finished.


In [21]:
# ------------------------------------------------------------------------
#                             VISUALIZATION
# ------------------------------------------------------------------------
# vis = Visualizer()
# open(vis)

# delete!(vis)
# mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf_file))
# set_configuration!(mvis, [0.0, 0.0])
# open(mvis)
MeshCatMechanisms.animate(mvis, ts, qs; realtimerate = 1.)

# animation = Animation(mvis, ts, qs)
# setanimation!(mvis, animation)

println("\n done.")


 done.
